In [132]:
from sys import argv
from os import chdir, path
import numpy as np
from oxDNA_analysis_tools import duplex_finder
from oxDNA_analysis_tools.UTILS.RyeReader import strand_describe, describe, get_confs, inbox

#set the working directory back to the root directory
chdir(path.dirname(path.abspath('')))

In [133]:
#each directory should have an input, topology, relaxed configuration and mean configuration with these names:
dirname = 'RNAog'

mean = 'mean.dat'
conf = dirname+'.dat'
top = dirname+'.top'
inp = 'input'

In [134]:
#Use oat to find all duplexes in the relaxed configuration (it won't work on mean)
chdir(dirname)
argv.clear()
argv.extend(['duplex_finder.py', inp, conf])

duplex_finder.main()

INFO: oxDNA_analysis_tools version: 2.0.0
INFO: running config.py installed at:  /home/erik/anaconda3/lib/python3.9/site-packages/oxDNA_analysis_tools/config.py
INFO: Python version: 3.9.7
INFO: Package Numpy found. Version: 1.20.3
INFO: No dependency issues found.
INFO: No outfile name provided, defaulting to "angles.txt"
/home/erik/anaconda3/lib/python3.9/site-packages/oxDNA_analysis_tools/UTILS/geom.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  t,c = np.linalg.lstsq(mat,y)[0]
INFO: Processing in blocks of 20 configurations
INFO: You can modify this number by running oat config -n <number>, which will be persistent between analyses.
INFO: Running oxRNA version DEC2013
INFO: Running modification of oxRNA with additional Debye-Huckel potential


Starting up 1 processes for 1 chunks
All spawned, waiting for results
--- 8298.687703847885 seconds ---



Please cite these publications for any work that uses the oxDNA simulation package
		- for the code:
			* P. Šulc et al., J. Chem. Phys. 137, 135101 (2012)
			* L. Rovigatti et al., J. Comput. Chem. 36, 1 (2015)
		- for the oxDNA model:
			* T. E. Ouldridge et al., J. Chem. Phys, 134, 085101 (2011)
		- for the oxDNA2 model:
			* B. E. K. Snodin et al., J. Chem. Phys. 142, 234901 (2015)
		- for the oxRNA model:
			* P. Šulc et al., J. Chem. Phys. 140, 235102 (2014)
INFO: Aggregated I/O statistics (set debug=1 for file-wise information)
	  0.000  B written to files
	  0.000  B written to stdout/stderr
INFO: Writing duplex data to angles.txt.  Use duplex_angle_plotter to graph data


In [135]:
#the previous block made angles.txt, read it into a dict associating particles to duplex ids
with open('angles.txt', 'r') as f:
    duplex_ends = f.readlines()
    
d1_ends = [d.split('\t')[2:4] for d in duplex_ends[1:]]
d2_ends = [d.split('\t')[4:6] for d in duplex_ends[1:]]
d1s = [list(range(int(d[0]), int(d[1])+1)) for d in d1_ends]
d2s = [list(range(int(d[0]), int(d[1])+1)) for d in d2_ends]

p_to_d = {}
for i, (d1, d2) in enumerate(zip(d1s, d2s)):
    for p in d1:
        p_to_d[p] = i
    for p in d2:
        p_to_d[p] = i+len(d1s)


In [136]:
# Get the topology info and the mean structure from files
top_info, traj_info = describe(None, mean)

mean_ox = get_confs(traj_info.idxs, traj_info.path, 0, 1, top_info.nbases)[0]
mean_ox = inbox(mean_ox, center=True)

In [137]:
# Compute the CoM for each domain in each duplex
d1_means = [np.mean(mean_ox.positions[idx], axis=0) for idx in d1s]
d2_means = [np.mean(mean_ox.positions[idx], axis=0) for idx in d2s]

d_means = [np.mean(mean_ox.positions[d1+d2], axis=0) for d1, d2 in zip(d1s, d2s)]

In [138]:
# Dump domain CoM and duplex CoM files to .xyz files
with open ("domain_centers.xyz", "w+") as f:
    f.write('\n'.join(' '.join(str(x) for x in y) for y in d1_means))
    f.write('\n')
    f.write('\n'.join(' '.join(str(x) for x in y) for y in d2_means))

with open ("duplex_centers.xyz", 'w+') as f:
    f.write('\n'.join(' '.join(str(x) for x in y) for y in d_means))

In [139]:
# Create a list of connections between domain centers in the format of an ANM parfile
# This was an initial attempt at visualization
# This makes two parfiles, one for the first strand (assmed to be the scaffold) and one for all others.
'''system, monomers = strand_describe(top)
backbone_par = []
backbone_par2 = []
bonded_par = []

for s in system:
    ds =  set({})
    for m in s:
        if m.id in p_to_d.keys():
            ds.add(p_to_d[m.id])
    ds = sorted(list(ds))

    for i, d in enumerate(ds[:-1]):
        backbone_par.append([d, ds[i+1]])

        if p_to_d[s[3].id] > len(d1s):
            backbone_par2.append([d - len(d1s), ds[i+1] - len(d1s)])
        else:
            backbone_par2.append([d, ds[i+1]])

for i, (d1, d2) in enumerate(zip(d1s, d2s)):
    bonded_par.append([i, i+len(d1s)])

'''

'system, monomers = strand_describe(top)\nbackbone_par = []\nbackbone_par2 = []\nbonded_par = []\n\nfor s in system:\n    ds =  set({})\n    for m in s:\n        if m.id in p_to_d.keys():\n            ds.add(p_to_d[m.id])\n    ds = sorted(list(ds))\n\n    for i, d in enumerate(ds[:-1]):\n        backbone_par.append([d, ds[i+1]])\n\n        if p_to_d[s[3].id] > len(d1s):\n            backbone_par2.append([d - len(d1s), ds[i+1] - len(d1s)])\n        else:\n            backbone_par2.append([d, ds[i+1]])\n\nfor i, (d1, d2) in enumerate(zip(d1s, d2s)):\n    bonded_par.append([i, i+len(d1s)])\n\n'

In [140]:
# Dump the connections for the scaffold as a parfile 
'''with open("backbone.par", "w+") as f:
    f.write(str(top_info.nbases)+'\n')
    for l in backbone_par:
        f.write(str(l[0])+' '+str(l[1])+' 1.0 s 5.0 1.0 1.0 1.0 1.0'+'\n')
'''

'with open("backbone.par", "w+") as f:\n    f.write(str(top_info.nbases)+\'\n\')\n    for l in backbone_par:\n        f.write(str(l[0])+\' \'+str(l[1])+\' 1.0 s 5.0 1.0 1.0 1.0 1.0\'+\'\n\')\n'

In [141]:
# Dump other connections as another parfile
'''with open("backbone2.par", "w+") as f:
    f.write(str(top_info.nbases)+'\n')
    for l in backbone_par2:
        f.write(str(l[0])+' '+str(l[1])+' 1.0 s 5.0 1.0 1.0 1.0 1.0'+'\n')
'''

'with open("backbone2.par", "w+") as f:\n    f.write(str(top_info.nbases)+\'\n\')\n    for l in backbone_par2:\n        f.write(str(l[0])+\' \'+str(l[1])+\' 1.0 s 5.0 1.0 1.0 1.0 1.0\'+\'\n\')\n'

In [142]:
'''# Connections between beads which are supposed to be bonded
with open('bonded.par', 'w+') as f:
    f.write(str(top_info.nbases)+'\n')
    for l in bonded_par:
        f.write(str(l[0])+' '+str(l[1])+' 1.0 s 5.0 1.0 1.0 1.0 1.0'+'\n')
'''

"# Connections between beads which are supposed to be bonded\nwith open('bonded.par', 'w+') as f:\n    f.write(str(top_info.nbases)+'\n')\n    for l in bonded_par:\n        f.write(str(l[0])+' '+str(l[1])+' 1.0 s 5.0 1.0 1.0 1.0 1.0'+'\n')\n"

Showed this to Petr but he doesn't like that the beads are overlapping even though this is the "right" visualization.  So the next attempt is going to be to draw the sphere at the backbone site of the middle nucleotide in the duplex.


In [143]:
# Find the median base pair in each duplex and write their IDs to a file
middle_nucs = [np.floor(np.median(d)) for d in d1s]
middle_nucs.extend([np.ceil(np.median(d)) for d in d2s])

with open('mid_n.txt', 'w+') as f:
    f.write(' '.join(str(int(x)) for x in middle_nucs))

In [144]:
# Use oat to extract just those nucleotides into a new dat/top file
from oxDNA_analysis_tools import subset_trajectory

argv.clear()
argv.extend(['subset_trajectory.py', mean, top, '-i', 'mid_n.txt', 'mid_n'])

subset_trajectory.main()

Starting up 1 processes for 1 chunks
All spawned, waiting for results


INFO: Processing in blocks of 20 configurations
INFO: You can modify this number by running oat config -n <number>, which will be persistent between analyses.
INFO: Wrote trajectories: ['mid_n.dat']
INFO: Wrote topologies: ['mid_n.top']


Then, go take the mid_n.dat and mid_n.top files to oxView.  Turn off centering when loading.

Run the following snippet to randomly generate some strand colors (only for the multi-stranded origamis):
```JavaScript
systems[0].strands.forEach(s => {
	c = view.getRandomHue()
	s.select()
	colorElements(c)
	clearSelection()
})
```

Run the following snippet to change the material and make the backbones big, transparent and pretty:
```JavaScript
switchMaterial(instanceMaterial2)
instanceMaterial2.transparent = true
instanceMaterial2.opacity = 0.7

systems[0].doVisuals(() => {
  let i = 0;
  i = systems[0].scales.indexOf(1);
  while(i != -1) {
      systems[0].scales[i] = 2;
      i = systems[0].scales.indexOf(1);
  }

  i = systems[0].bbconScales.indexOf(1);
  while(i != -1) {
      systems[0].bbconScales[i] = 1.5;
      i = systems[0].bbconScales.indexOf(1);
  }

  i = systems[0].conScales.indexOf(1);
  while(i != -1) {
      systems[0].conScales[i] = 2;
      i = systems[0].conScales.indexOf(1);
  }
})

scene.remove(systems[0].nucleoside)

render()
```
Take a screenshot.

Then without moving the camera load up the mean structure, select the scaffold, color it a light grey (the other strands will jump to the `GREY` color)

Run
```JavaScript
api.toggleBaseColors()
```
to remove the base colors.

Take another screenshot.